# Active Learning with Pixano - MNIST Dataset

## initialization of various parameters

### Configuration variables

In [1]:
# The root dir name of the current repo (i.e. pixano or pixano-main etc.)
ROOTDIR='pixano'
# directory where Mnist dataset is downloaded using Notebook "1.convert_dataset_2_explorer.ipynb"
datasets_dir="/home/melissap/Desktop/LAGO/3.githubs/integration/datasets/MNIST" 
# name of the dataset as defined in "1.convert_dataset_2_explorer.ipynb"
DATASET_NAME="MNIST_pixano_v1"
# directory where the raw mnist dataset will be saved to be transformed latter (images), and also to be used by the active learning auto-annotator (labels)
pixano_datasets_dir='/home/melissap/_pixano_datasets_'
# conda env name builded for running the active learning module as a separate program
customLearnerCondaEnv="customLearner"
# ActiveLearning module's directory
ALModule="ActiveLearning"

### internal experimental variables (that could be defined by the user)

In [2]:
labels_per_round=100
numInitLabels = labels_per_round
learning_rate=0.001
max_epochs_per_round=100
model_name="mlp" 
strategy="AlphaMixSampling" #i.e. other alternatives may be : EntropySampling #RandomSampling
alpha_opt=True

### External experimental variables

In [3]:
num_rounds = 10

## ... the rest of the notebook should run without any code adjustments/modifications.

> import ROOT dir to import pixano root module , which is the pixano directory

In [4]:
import os
import sys

def insertRootDir(ROOTDIR='pixano'):
    pardir=os.path.dirname(os.path.realpath('__file__'))

    found = False
    potential_root_dir = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath('__file__'))))))

    while(os.path.basename(pardir)!=ROOTDIR):

        # print(pardir)
        pardir=os.path.dirname(pardir)
        

        if (os.path.basename(pardir) == ROOTDIR):
            found = True
            break
        if (pardir == "/" ):
            break
    
    if found:
        print("Inserting parent dir : ",pardir)
        sys.path.insert(0,pardir)
        return pardir
    else:
        print(f"ROOTDIR NOT FOUND. You may have to change ROOTDIR variable from : '{ROOTDIR}' to '{potential_root_dir}'")
        return "_NOT_FOUND_"

ROOTDIR = insertRootDir(ROOTDIR)

Inserting parent dir :  /home/melissap/Desktop/LAGO_43integrationDemo/pixano


In [5]:
ALModuleDir = os.path.join(ROOTDIR,ALModule)
print(f"ALModuleDir {ALModuleDir}")

ALModuleDir /home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning


In [6]:
from pathlib import Path
from pixano.data import ImageImporter

In [7]:
library_dir=Path(pixano_datasets_dir) # directory where we have install the pixano formatted dataset
import_dir = library_dir / DATASET_NAME

In [8]:
import random
import lancedb
import pyarrow as pa
import numpy as np
import pandas as pd
from PIL import Image
from ActiveLearning.ALearner import (
    Learner,
    BaseAnnotator,
    BaseSampler,
    BaseTrainer,
    getLabels,
    getLabelledIds,
    getUnlabelledIds,
    getTaggedIds,
    getLastRound,
    ddb_str,
    custom_update,
    importTestLabels
)
from pixano.utils import natural_key

In [9]:
from ActiveLearning.customTrainer import customTrainer
from ActiveLearning.customSampler import customSampler

In [10]:
# utility function to convert id (format "<index>.png") to index
def id_to_idx(id: str) -> int:
    return int(id.split(".")[0])
    # return int(id[0:-4])  #remove the last 4 chars (".png")

### Connect to Pixano DB
MNIST dataset should have been imported previously (see lance_importers/MNIST.ipynb)

In [11]:
mnist_db = lancedb.connect(import_dir)

## Model Trainer Object

We will get raw x_train, x_test, y_test data directly from MNIST.

2 proposed Model Trainer Objects, with same model: SimpleTrainer and IncrementalTrainer

In [12]:
def get_MNIST(data_dir):

    image_dir = os.path.join(data_dir,"images")
    train_imdir = os.path.join(image_dir,"train")
    test_imdir = os.path.join(image_dir,"test")

    annotation_dir = os.path.join(data_dir,"annotations")
    train_anfile = os.path.join(annotation_dir,"train.csv")
    test_anfile = os.path.join(annotation_dir,"test.csv")

    # read sorted file names     
    X_train = sorted(os.listdir(train_imdir), key=lambda x: int(x.split('.', 1)[0])) 
    X_test = sorted(os.listdir(test_imdir), key=lambda x: int(x.split('.', 1)[0]))
    
    # X_train = np.array([Image.open(os.path.join(train_imdir,x)) for x in X_train])
    # X_test = np.array([Image.open(os.path.join(test_imdir,x)) for x in X_test])

    # # read labels
    Y_train = pd.read_csv(train_anfile).values #.to_numpy(dtype=np.uint8)
    Y_test = pd.read_csv(test_anfile).values #.to_numpy(dtype=np.uint8)

    Y_train = np.array([y[0] for y in Y_train], dtype=np.uint8)
    Y_test = np.array([y[0] for y in Y_test], dtype=np.uint8)

    return (X_train,Y_train),(X_test,Y_test)

# (xt,yt),(xte,yte) = get_MNIST(datasets_dir)
(X_train, Y_train), (X_test, Y_test) = get_MNIST(datasets_dir)

## Query Sampler Object
<!-- RandomSampler or SequentialSampler -->

#### Custom Trainer

> prepare the directories for data exchange between pixano and annotation tool

In [13]:
import shutil

# TEMPORARY SOLUTION FOR EXCHANGING DATA BETWEEN PIXANO AND AL
def create_dir(path):
    try:
        if (os.path.basename(path)== "temp_data" and os.path.exists(path)):
            shutil.rmtree(path) # erase the previous results
        os.makedirs(path)
    except:
        print(f'Dir {path} exists already')
    return path

# here define the paths of exchanging data between pixano and the customLearner
temp_data_exchange_dir = create_dir(os.path.join(ROOTDIR,"temp_data"))                # define a directory for exchanging data
output_queDir = create_dir(os.path.join(temp_data_exchange_dir,"output_queries"))       # [out] query strategy results
output_accDir = create_dir(os.path.join(temp_data_exchange_dir,"output_accuracy"))      # [out] accuracy results 

## Labeling Interface Objects

Human labeling with Pixano Annotator is built-in, here we specify an Auto Annotator

In [14]:
class AutoAnnotator(BaseAnnotator):
    # custom annotation function
    # as we have ground truth for MNIST, we can autofill
    def annotate(self, round):
        candidates = getTaggedIds(self.db, round)
        db_tbl = mnist_db.open_table("db")
        custom_update(db_tbl, f"id in ({ddb_str(candidates)})", 'label', [str(Y_train[id_to_idx(candidate)]) for candidate in sorted(candidates, key=natural_key)])
        print(f"AutoAnnotator: round {round} annotated.")

## Orchestrator

### Initial Learning

In [15]:
myTrainer = customTrainer(mnist_db, 
                            DATASET_NAME = DATASET_NAME,
                            output_accDir = output_accDir,
                            import_dir = import_dir,
                            customLearnerCondaEnv = customLearnerCondaEnv,
                            model_name = model_name, 
                            learning_rate = learning_rate, 
                            max_epochs_per_round = max_epochs_per_round,
                            strategy_name = strategy)

mycustomSampler = customSampler(mnist_db,
                                DATASET_NAME = DATASET_NAME,
                                output_queDir = output_queDir,
                                import_dir = import_dir,
                                customLearnerCondaEnv = customLearnerCondaEnv,
                                model_name = model_name, 
                                strategy_name = strategy, 
                                number_init_labels = numInitLabels, 
                                labels_per_round = labels_per_round,
                                alpha_opt = True)

myTrainer.set_parameter("ALearner",ALModuleDir)
mycustomSampler.set_parameter("ALearner",ALModuleDir)

autofillAnnotator = AutoAnnotator(mnist_db)

init_learner = Learner(
    db=mnist_db,
    trainer=myTrainer,
    sampler=mycustomSampler,
    custom_annotator=autofillAnnotator,
    new_al=True,
    verbose=0
)

In [16]:
importTestLabels(mnist_db,(X_test,Y_test))

Importing validation data in /home/melissap/_pixano_datasets_/MNIST_pixano_v1


### Active Learning - Human annotation with Pixano Annotator

We add some auto-annotation rounds

In [17]:
for round in range(num_rounds):
    candidates = init_learner.query(round)
    init_learner.annotate(round)
    init_learner.train(round, epochs="nevermind")

csvQue /home/melissap/Desktop/LAGO_43integrationDemo/pixano/temp_data/output_queries/queries_0.csv


######################################################## AL ROUND 0 mode query : START #########################################################


Namespace(mode='query', data_name='MNIST_pixano_v1', n_label='None', n_init_lb=100, n_query=100, data_dir='/home/melissap/_pixano_datasets_/MNIST_pixano_v1', pixano_al_module='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning', strategy='AlphaMixSampling', round=0, train_out=None, query_out='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/temp_data/output_queries/queries_0.csv', seeds=[1, 10, 100, 1000, 10000], init_lb_method='general_random', log_dir='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning/certh_integration/alpha_mix_active_learning/_logs', save_checkpoints=False, save_images=False, print_to_file=False, query_growth_ratio=1, n_drop=5, eps=0.05, max_iter=50, alpha_cap=0.03125, alpha_opt=True

 10%|█         | 10/100 [00:05<00:41,  2.19it/s]

Reached max accuracy at epoch 10 


 10%|█         | 10/100 [00:05<00:50,  1.77it/s]


Round 0
testing accuracy 0.7615


######################################################### AL ROUND 0 mode train : END ##########################################################


csvQue /home/melissap/Desktop/LAGO_43integrationDemo/pixano/temp_data/output_queries/queries_1.csv


######################################################## AL ROUND 1 mode query : START #########################################################


Namespace(mode='query', data_name='MNIST_pixano_v1', n_label='None', n_init_lb=100, n_query=100, data_dir='/home/melissap/_pixano_datasets_/MNIST_pixano_v1', pixano_al_module='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning', strategy='AlphaMixSampling', round=1, train_out=None, query_out='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/temp_data/output_queries/queries_1.csv', seeds=[1, 10, 100, 1000, 10000], init_lb_method='general_random', log_dir='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning/certh_integration/alpha

...

Round 9 tagged
100 candidates on round 9
AutoAnnotator: round 9 annotated.


######################################################## AL ROUND 9 mode train : START #########################################################


Namespace(mode='train', data_name='MNIST_pixano_v1', n_label='None', n_init_lb=10, n_query=100, data_dir='/home/melissap/_pixano_datasets_/MNIST_pixano_v1', pixano_al_module='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning', strategy='AlphaMixSampling', round=9, train_out='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/temp_data/output_accuracy/accuracy.csv', query_out='path to the file', seeds=[1, 10, 100, 1000, 10000], init_lb_method='general_random', log_dir='/home/melissap/Desktop/LAGO_43integrationDemo/pixano/ActiveLearning/certh_integration/alpha_mix_active_learning/_logs', save_checkpoints=False, save_images=False, print_to_file=False, query_growth_ratio=1, n_drop=5, eps=0.05, max_iter=50, alpha_cap=0.03125, alpha_opt=False, alpha_cl

  2%|▏         | 2/100 [00:01<01:24,  1.17it/s]

Reached max accuracy at epoch 2 


  2%|▏         | 2/100 [00:02<01:53,  1.16s/it]


Round 0
testing accuracy 0.9377


######################################################### AL ROUND 9 mode train : END ##########################################################


